## A combinação NumPy, Pandas e Scikit-Learn é muito poderosa

Embora O Scikit-Learn seja responsável por facilitar grande parte do pipeline de Machine Learning, dois componentes são essenciais para ler arquivos e manipular as informações: NumPy e Pandas. O NumPy auxilia a manipulação matricial dos dados, que é fundamental para aprendizado de máquina; enquanto o Pandas atua como um facilitador para a manipulação de dados de forma menos estruturada.

*Esse conteúdo é baseado no capítulo 'Machine Learning' de Numerical Python, segunda edição, de Robert Johannson.*

A seguir é utilizado o comando read_csv do Pandas para ter acesso aos dados do arquivo 'energy_data.csv', e o comando *describe* gera um sumário dos dados quando possível. A nomenclatura df para a variável vem de DataFrame do Pandas, mas poderia ser o nome que o programador quisesse dar.

In [ ]:
import pandas as pd

df = pd.read_csv('../input/appliances-energy-prediction/KAG_energydata_complete.csv')
df.describe()

Esse conjunto de dados está disponível em https://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction# e um Github do artigo que originou essa contribuição está disponível em https://github.com/LuisM78/Appliances-energy-prediction-data.

O objetivo é utilizar as informações de clima fora e dentro de uma casa para estimar o consumo de energia elétrico que aquela casa, naquelas condições meteorológicas, demandaria. Portanto, serão considerados atributos todas as colunas exceto appliances e a data, e essa coluna será utilizada como *target*.

In [ ]:
X = df.drop(['Appliances','date'],axis=1)
y = df.Appliances

In [ ]:
print(X.shape)
print(y.shape)

São 27 atributos que farão a composição de X, e o total de amostras são 19735. Por se tratar de um problema de regressão, serão importados os módulos do Scikit-Learn para a separação dos dados e a geração dos regressores (modelos de regressão). Em seguida, os dados são separados com o comando *train_test_split* e um modelo de Regressão Linear é treinado com a função *fit*.

**Vamos supor um cenário com 20 amostras do conjunto inicial para propósitos didáticos:**

In [ ]:
from sklearn import model_selection
from sklearn import linear_model

X_train, X_test, y_train, y_test = model_selection.train_test_split(X[:20], y[:20], test_size=0.5, random_state=42)
model = linear_model.LinearRegression()

model.fit(X_train, y_train)

Em seguida, é importante calcular o resíduo entre o que o modelo diz e o que de fato é. Esse resíduo também pode ser acumulado como erro utilizando alguma função. Vamos verificar então qual é o erro para o próprio conjunto de treino utilizando o erro quadrático médio, e depois o erro no teste.

In [ ]:
from sklearn import metrics
import numpy as np

In [ ]:
erro_treino = metrics.mean_squared_error(y_train,model.predict(X_train))
print('RMSE no treino:', np.sqrt(erro_treino))

erro_teste = metrics.mean_squared_error(y_test,model.predict(X_test))
print('RMSE no teste:', np.sqrt(erro_teste))

É possível perceber que o erro no treino é muito baixo, praticamente zero, enquanto que o erro no teste é bastante alto (> 104). Isso se deve a um fenômeno chamado **overfitting**. É muito comum que, quando o número de atributos seja maior que o número de amostras, a regressão fique sobreajustada (overfit) para os dados de treino, e não generaliza suficiente para o conjunto de teste.

In [ ]:
r2 = model.score(X_train, y_train)
print('r² no treino:', r2)

r2 = model.score(X_test, y_test)
print('r² no teste:', r2)

Por último, também é possível visualizar essa diferença de forma gráfica. Compara-se o resíduo por amostra tanto no treino quanto no teste, e observa-se os coeficientes encontrados:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns

In [ ]:
def plot_residuals_and_coeff(resid_train, resid_test, coeff):
    fig, axes = plt.subplots(1, 3, figsize=(12, 3))
    axes[0].bar(np.arange(len(resid_train)), resid_train)
    axes[0].set_xlabel("núm. amostras")
    axes[0].set_ylabel("resíduo")
    axes[0].set_title("treino")
    axes[1].bar(np.arange(len(resid_test)), resid_test)
    axes[1].set_xlabel("núm. amostras")
    axes[1].set_ylabel("resíduo")
    axes[1].set_title("teste")
    axes[2].bar(np.arange(len(coeff)), coeff)
    axes[2].set_xlabel("núm. coeficientes")
    axes[2].set_ylabel("coeficiente")
    fig.tight_layout()
    return fig, axes

residuo_treino = y_train - model.predict(X_train)
residuo_teste  = y_test - model.predict(X_test)

fig, ax = plot_residuals_and_coeff(residuo_treino, residuo_teste, model.coef_)

É possível perceber nas escalas dos resíduos que os erros são muito maiores no conjunto de teste do que no conjunto de treino. *Uma das formas de corrigir esse problema é aplicando regularização*, forçando os coeficientes a residirem num espaço próximo. As duas normais mais comuns de regularização são L1 e L2, respectivamente LASSO e Ridge. Enquanto que L2 favorece com coeficientes menores, L1 favorece modelos que têm poucos coeficientes próximos de zero.


**Quando usar L1 ou L2?**

* L1: Deseja-se eliminar o maior número de atributos que não contribuem com o problema;
* L2: Limitar a magnitude dos coeficientes do modelo.

**Utilizando Ridge — L2**

In [ ]:
model = linear_model.Ridge()
model.fit(X_train, y_train)

erro_treino = metrics.mean_squared_error(y_train,model.predict(X_train))
print('RMSE no treino:', np.sqrt(erro_treino))

erro_teste = metrics.mean_squared_error(y_test,model.predict(X_test))
print('RMSE no teste:', np.sqrt(erro_teste))

O erro no treino não é mais próximo a zero, mas é possível perceber que *o erro no teste diminuiu*. Vale lembrar que os dados do teste são mais parecidos com os dados do mundo real, ou seja, são informações que o modelo não teve acesso, portanto mais difíceis de serem previstas corretamente.

Graficamente, é possível perceber uma mudança nos pesos também:

In [ ]:
residuo_treino = y_train - model.predict(X_train)
residuo_teste  = y_test - model.predict(X_test)

fig, ax = plot_residuals_and_coeff(residuo_treino, residuo_teste, model.coef_)

**Utilizando LASSO — L1**

In [ ]:
model = linear_model.Lasso(alpha=2.5)
model.fit(X_train, y_train)

erro_treino = metrics.mean_squared_error(y_train,model.predict(X_train))
print('RMSE no treino:', np.sqrt(erro_treino))

erro_teste = metrics.mean_squared_error(y_test,model.predict(X_test))
print('RMSE no teste:', np.sqrt(erro_teste))

In [ ]:
residuo_treino = y_train - model.predict(X_train)
residuo_teste = y_test - model.predict(X_test)

fig, ax = plot_residuals_and_coeff(residuo_treino, residuo_teste, model.coef_)

Os resultados de erro no treino são mais baixos que do Ridge, porém mais altos que a regressão linear sem regularização, porém o erro no teste caiu aproximadamente para a metade. Também é possível perceber que L1 reduziu significativamente o número de atributos que, de fato, contribuem para a solução do problema.

Também é possível perceber que o **parâmetro alpha é configurável**, portanto é importante fazer uma varredura para identificar qual seria o melhor valor.

## Exercícios

1. **Faça a busca de melhor parâmetro de alpha para reduzir o erro quadrático médio nessa base de apenas 20 amostras, considerando L1 e L2. Considere utilizar as funções LassoCV e RidgeCV, e busque informações sobre elas na documentação do Scikit-Learn.**


2. **Recarregue a base e não filtre apenas 20 amostras e aplique os modelos de regressão vistos. Tenha um olhar crítico para os seguintes questionamentos após codificar e ver os resultados:**
    * Qual regressão teve o melhor resultado?
    * Há algum sinal de overfitting?
    * Todos atributos são relevantes para o problema?
    * Todos coeficientes estão em uma mesma magnitude?

In [ ]:
df = pd.read_csv('../input/appliances-energy-prediction/KAG_energydata_complete.csv')
X_train, X_test, y_train, y_test = model_selection.train_test_split(X[:20], y[:20], test_size=0.5, random_state=42)

rmse_treino = []
rmse_teste  = []
alpha = []

for a in range(-4,8):
    model = linear_model.Lasso(alpha=10**a)
    model.fit(X_train, y_train)
    
    print("################################################")
    print('Alpha:', 10**a)
    
    rmse_treino.append(np.sqrt(metrics.mean_squared_error(y_train, model.predict(X_train))))
    rmse_teste.append(np.sqrt(metrics.mean_squared_error(y_test, model.predict(X_test))))
    alpha.append(a)
    
    print('MSE no treino:', rmse_treino[-1])
    print('MSE no teste:', rmse_teste[-1])
    
    print("################################################")

plt.plot(alpha, rmse_treino, alpha, rmse_teste)

In [ ]:
rmse_treino = []
rmse_teste  = []
alpha = []

for a in range(-4,8):
    model = linear_model.Ridge(alpha=10**a)
    model.fit(X_train, y_train)
    
    print("################################################")
    print('Alpha:', 10**a)
    
    rmse_treino.append(np.sqrt(metrics.mean_squared_error(y_train, model.predict(X_train))))
    rmse_teste.append(np.sqrt(metrics.mean_squared_error(y_test, model.predict(X_test))))
    alpha.append(a)
    
    print('MSE no treino:', rmse_treino[-1])
    print('MSE no teste:', rmse_teste[-1])
    
    print("################################################")

plt.plot(alpha, rmse_treino, alpha, rmse_teste)

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV

In [ ]:
# Utilizando o Lasso CV
lasso_reg = LassoCV(cv=5, random_state=0)
lasso_reg.fit(X_train, y_train)
best_alpha_lasso = lasso_reg.alpha_
print("Best alpha => ", best_alpha_lasso)
# Com este valor fazemos o cross validation para outros parâmetros

In [ ]:
# Utilizando o Ridge CV
ridge_reg = RidgeCV(cv=5)
ridge_reg.fit(X_train, y_train)
best_alpha_ridge = ridge_reg.alpha_
print("Best alpha => ", best_alpha_ridge)
# Com este valor fazemos o cross validation para outros parâmetros

Podemos agora carregar todos os dados e comparar o desempenho utilizando L1 e L2

In [ ]:
X = df.drop(['Appliances','date'],axis=1)
y = df.Appliances

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.5, random_state=42)

model = linear_model.Ridge(alpha = best_alpha_ridge)
model.fit(X_train, y_train)

erro_treino = np.sqrt(metrics.mean_squared_error(y_train,model.predict(X_train)))
print('RMSE no treino:', erro_treino)

erro_teste = np.sqrt(metrics.mean_squared_error(y_test,model.predict(X_test)))
print('RMSE no teste:', erro_teste)

residuo_treino = y_train - model.predict(X_train)
residuo_teste  = y_test - model.predict(X_test)

fig, ax = plot_residuals_and_coeff(residuo_treino, residuo_teste, model.coef_)

In [ ]:
X = df.drop(['Appliances','date'],axis=1)
y = df.Appliances

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.5, random_state=42)

model = linear_model.Lasso(alpha = best_alpha_lasso)
model.fit(X_train, y_train)

erro_treino = np.sqrt(metrics.mean_squared_error(y_train,model.predict(X_train)))
print('RMSE no treino:', erro_treino)

erro_teste = np.sqrt(metrics.mean_squared_error(y_test,model.predict(X_test)))
print('RMSE no teste:', erro_teste)

residuo_treino = y_train - model.predict(X_train)
residuo_teste  = y_test - model.predict(X_test)

fig, ax = plot_residuals_and_coeff(residuo_treino, residuo_teste, model.coef_)

Em vez de utilizarmos o LassoCV ou RidgeCV, poderíamos também buscar por eles através do grid search

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

data_gs, data_cv, target_gs, target_cv = train_test_split(X, y, test_size=0.95, random_state=42)

pipeline = Pipeline([('scaler', StandardScaler()), ('clf', linear_model.Ridge())])

# utiliza-se GridSearchCV para achar os melhores parâmetros
from sklearn.model_selection import GridSearchCV
parameters = {'clf__alpha': [0.1,1, 1.5, 3, 5, 7,10,100,1000]} # quais parâmetros e quais valores serão testados
clf = GridSearchCV(pipeline, parameters, cv=3, iid=False) # clf vai armazenar qual foi a melhor configuração
clf.fit(data_gs, target_gs)

print(clf.best_params_ )

# utilizando validação cruzada para avaliar o modelo
scores = cross_val_score(clf, data_cv, target_cv, cv=5, scoring='neg_mean_squared_error')

scores = -scores
scores = np.sqrt(scores)

print('RMSE - %.2f +- %.2f' % (scores.mean(), scores.std()))

In [ ]:
print('R²:', clf.score(X_test, y_test) )
erro_teste = np.sqrt(metrics.mean_squared_error(y_test, clf.predict(X_test)))
print('RMSE no treino:', erro_teste)

Testando agora o Lasso

In [ ]:
data_gs, data_cv, target_gs, target_cv = train_test_split(X, y, test_size=0.95, random_state=42)

pipeline = Pipeline([('scaler', StandardScaler()), ('clf', linear_model.Lasso())])

# utiliza-se GridSearchCV para achar os melhores parâmetros
from sklearn.model_selection import GridSearchCV
parameters = {'clf__alpha': [0.1,1, 1.5, 3, 5, 7,10,100,1000]} # quais parâmetros e quais valores serão testados
clf = GridSearchCV(pipeline, parameters, cv=3, iid=False) # clf vai armazenar qual foi a melhor configuração
clf.fit(data_gs, target_gs)

print(clf.best_params_ )

# utilizando validação cruzada para avaliar o modelo
scores = cross_val_score(clf, data_cv, target_cv, cv=5, scoring='neg_root_mean_squared_error')

scores = -scores
# scores = np.sqrt(scores)

print('RMSE - %.2f +- %.2f' % (scores.mean(), scores.std()))

In [ ]:
print('R²:', clf.score(X_test, y_test) )
erro_teste = np.sqrt(metrics.mean_squared_error(y_test, clf.predict(X_test)))
print('RMSE no treino:', erro_teste)

Elasticnet combina L1 e L2

In [ ]:
data_gs, data_cv, target_gs, target_cv = train_test_split(X, y, test_size=0.95, random_state=42)

pipeline = Pipeline([('scaler', StandardScaler()), ('clf', linear_model.ElasticNet())])

# utiliza-se GridSearchCV para achar os melhores parâmetros
from sklearn.model_selection import GridSearchCV
parameters = {'clf__alpha': [0.1,1, 1.5, 3, 5, 7,10,100,1000]} # quais parâmetros e quais valores serão testados
clf = GridSearchCV(pipeline, parameters, cv=3, iid=False) # clf vai armazenar qual foi a melhor configuração
clf.fit(data_gs, target_gs)

print(clf.best_params_ )

# utilizando validação cruzada para avaliar o modelo
scores = cross_val_score(clf, data_cv, target_cv, cv=5, scoring='neg_mean_squared_error')

scores = -scores
scores = np.sqrt(scores)

print('RMSE - %.2f +- %.2f' % (scores.mean(), scores.std()))

In [ ]:
print('R²:', clf.score(X_test, y_test) )
erro_teste = np.sqrt(metrics.mean_squared_error(y_test, clf.predict(X_test)))
print('RMSE no treino:', erro_teste)

In [ ]:
residuo_treino = y_train - clf.predict(X_train)
residuo_teste  = y_test - clf.predict(X_test)

fig, ax = plot_residuals_and_coeff(residuo_treino, residuo_teste, clf.best_estimator_['clf'].coef_)